In [1]:
import numpy as np
import pandas as pd 
import requests 
from scipy import stats 
import math

In [2]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

df = pd.read_csv('S&P500.csv')
df.head()

,Symbol
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [3]:
Companies = {}  # stores company names along with mkt cap and latest price
Companies['Name'] = []
Companies['CMP'] = []
Companies['1 year change'] = []


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
batches  = list(chunks(df['Symbol'], 100)) # 

for i in range(len(batches)):
    batch = (','.join(batches[i]))
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for company in batch.split(','):
        Companies['Name'].append(company)
        Companies['CMP'].append(data[company]['quote']['latestPrice'])
        Companies['1 year change'].append(data[company]['stats']['year1ChangePercent'])
        

df = pd.DataFrame(Companies)
df

,Name,CMP,1 year change
0,A,127.13,-0.174710
1,AAL,14.20,-0.389869
2,AAP,189.60,-0.066549
3,AAPL,147.07,0.068332
4,ABBV,158.17,0.393122
...,...,...,...
497,YUM,119.25,-0.004964
498,ZBH,112.21,-0.337914
499,ZBRA,323.92,-0.404897
500,ZION,53.41,-0.016967


In [4]:
df = df.sort_values(by = '1 year change' , ascending = False)
df

,Name,CMP,1 year change
148,DVN,54.77,0.959554
354,OXY,59.10,0.841403
296,MCK,330.81,0.731161
90,CF,83.42,0.688159
40,APA,37.38,0.654799
...,...,...,...
295,MCHP,62.66,-0.616600
84,CCL,11.36,-0.621134
328,NFLX,194.48,-0.644189
207,GPS,9.79,-0.730412


In [5]:
top50 = df.head(50)
top50.reset_index(inplace = True)

In [6]:
top50.head(5)

,index,Name,CMP,1 year change
0,148,DVN,54.77,0.959554
1,354,OXY,59.10,0.841403
2,296,MCK,330.81,0.731161
3,90,CF,83.42,0.688159
4,40,APA,37.38,0.654799


In [7]:
portfolio_size = 100000

In [8]:
# shares to buy 
# allocating equal amount for each share

# Shares for each company
amount = portfolio_size/len(top50['Name'])

pd.set_option('mode.chained_assignment', None)
top50['Shares to buy'] = 0
for i in range(len(top50['Name'])):
    top50['Shares to buy'][i] = math.floor(amount/top50['CMP'][i])
    
top50

,index,Name,CMP,1 year change,Shares to buy
0,148,DVN,54.77,0.959554,36
1,354,OXY,59.10,0.841403,33
2,296,MCK,330.81,0.731161,6
3,90,CF,83.42,0.688159,23
4,40,APA,37.38,0.654799,53
5,315,MRO,22.84,0.639614,87
6,139,DLTR,160.54,0.611519,12
7,119,CTRA,27.52,0.608069,72
8,109,COP,95.02,0.549019,21
9,471,VRTX,303.89,0.525280,6


In [9]:
#filtering high quality momentum stocks -> steady returns
#low quality momtum stocks -> sudden jump
df1 = pd.read_csv('S&P500.csv')

Companies = {}  # stores company names along with mkt cap and latest price
Companies['Name'] = []
Companies['CMP'] = []
Companies['6 month change'] = []
Companies['6 month %ile'] = []
Companies['1 year change'] = []
Companies['1 year %ile'] = []
Companies['2 year change'] = []
Companies['2 year %ile'] = []
Companies['momentum score'] = []

batches  = list(chunks(df1['Symbol'], 100)) 

for i in range(len(batches)):
    batch = (','.join(batches[i]))
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for company in batch.split(','):
        Companies['Name'].append(company)
        Companies['CMP'].append(data[company]['quote']['latestPrice'])
        Companies['1 year change'].append(data[company]['stats']['year1ChangePercent'])
        Companies['2 year change'].append(data[company]['stats']['year2ChangePercent'])
        Companies['6 month change'].append(data[company]['stats']['month6ChangePercent'])
        Companies['6 month %ile'].append(0)
        Companies['1 year %ile'].append(0)
        Companies['2 year %ile'].append(0)
        Companies['momentum score'].append(0)

df1 = pd.DataFrame(Companies)
df1

,Name,CMP,6 month change,6 month %ile,1 year change,1 year %ile,2 year change,2 year %ile,momentum score
0,A,121.97,-0.239170,0,-0.174170,0,0.427574,0,0
1,AAL,14.10,-0.247118,0,-0.381719,0,0.068633,0,0
2,AAP,186.70,-0.202120,0,-0.066337,0,0.337403,0,0
3,AAPL,143.82,-0.200199,0,0.068608,0,0.621211,0,0
4,ABBV,157.42,0.168435,0,0.401501,0,0.775049,0,0
...,...,...,...,...,...,...,...,...,...
497,YUM,119.63,-0.148850,0,-0.004949,0,0.376433,0,0
498,ZBH,110.00,-0.141301,0,-0.349339,0,-0.068376,0,0
499,ZBRA,309.45,-0.492327,0,-0.410664,0,0.237990,0,0
500,ZION,55.72,-0.148214,0,-0.016544,0,0.629431,0,0


In [10]:
for i in range(len(df1['Name'])):
    df1['6 month %ile'][i] = stats.percentileofscore(df1['6 month change'],df1['6 month change'][i])/100
    df1['1 year %ile'][i] = stats.percentileofscore(df1['1 year change'],df1['1 year change'][i])/100
    df1['2 year %ile'][i] = stats.percentileofscore(df1['2 year change'],df1['2 year change'][i])/100
    df1['momentum score'][i] = (df1['6 month %ile'][i]+df1['1 year %ile'][i]+df1['2 year %ile'][i])/3

df1.head()

,Name,CMP,6 month change,6 month %ile,1 year change,1 year %ile,2 year change,2 year %ile,momentum score
0,A,121.97,-0.239170,0.306773,-0.174170,0.308765,0.427574,0.581673,0.399070
1,AAL,14.10,-0.247118,0.284861,-0.381719,0.083665,0.068633,0.199203,0.189243
2,AAP,186.70,-0.202120,0.398406,-0.066337,0.529880,0.337403,0.478088,0.468792
3,AAPL,143.82,-0.200199,0.406375,0.068608,0.729084,0.621211,0.741036,0.625498
4,ABBV,157.42,0.168435,0.926295,0.401501,0.962151,0.775049,0.834661,0.907703


In [11]:
# now selecting top 50 based on momentum score
df1 = df1.sort_values(by = 'momentum score' , ascending = False)
top50 = df1.head(50)
top50.reset_index(inplace = True)
top50.head()

,index,Name,CMP,6 month change,6 month %ile,1 year change,1 year %ile,2 year change,2 year %ile,momentum score
0,354,OXY,57.93,1.038877,1.000000,0.857945,0.998008,2.224646,0.988048,0.995352
1,148,DVN,55.11,0.330034,0.980080,0.976500,1.000000,4.522603,1.000000,0.993360
2,315,MRO,22.74,0.402305,0.990040,0.642162,0.990040,2.971370,0.994024,0.991368
3,296,MCK,322.43,0.349300,0.982072,0.744792,0.996016,1.210439,0.948207,0.975432
4,385,PXD,227.60,0.309147,0.974104,0.497365,0.974104,1.710254,0.976096,0.974768


In [12]:
portfolio_size = 100000


pd.set_option('mode.chained_assignment', None)
top50['Shares to buy'] = 0
for i in range(len(top50['Name'])):
    top50['Shares to buy'][i] = math.floor(amount/top50['CMP'][i])
    
top50

,index,Name,CMP,6 month change,6 month %ile,1 year change,1 year %ile,2 year change,2 year %ile,momentum score,Shares to buy
0,354,OXY,57.93,1.038877,1.000000,0.857945,0.998008,2.224646,0.988048,0.995352,34
1,148,DVN,55.11,0.330034,0.980080,0.976500,1.000000,4.522603,1.000000,0.993360,36
2,315,MRO,22.74,0.402305,0.990040,0.642162,0.990040,2.971370,0.994024,0.991368,87
3,296,MCK,322.43,0.349300,0.982072,0.744792,0.996016,1.210439,0.948207,0.975432,6
4,385,PXD,227.60,0.309147,0.974104,0.497365,0.974104,1.710254,0.976096,0.974768,8
5,311,MPC,85.92,0.381575,0.984064,0.421246,0.966135,1.567538,0.970120,0.973440,23
6,494,XOM,87.10,0.455464,0.996016,0.422014,0.968127,1.238935,0.956175,0.973440,22
7,310,MOS,48.80,0.198182,0.944223,0.509979,0.978088,2.990691,0.996016,0.972776,40
8,109,COP,94.17,0.307683,0.972112,0.534280,0.982072,1.424754,0.964143,0.972776,21
9,211,HAL,32.33,0.384048,0.986056,0.321746,0.946215,1.656418,0.972112,0.968127,61
